In [1]:
import numpy as np
import DataLoader,DomainManipulation
import math
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from hmmlearn.hmm import GMMHMM
import librosa
import time
import mdp
from sklearn.decomposition import PCA

C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
traindir = 'data/train_audio'
targetdir = 'data/birdseperation-sfa'

In [2]:
start_time = time.time()

dirs = os.listdir(traindir)
dirs = dirs[:1]

# for birddir in dirs:
birddir = dirs[0]
files = os.listdir(os.path.join(traindir,birddir))

sfa_components = []
sfa_file_name_order = []
hmm_sfa_corr = []
for fileName in files:
    birdLoc = os.path.join(traindir,birddir,fileName)
    targetLoc = os.path.join(targetdir,birddir,fileName)
    os.makedirs(targetLoc, exist_ok=True)

    try:
        y = DataLoader.readFromFile(birdLoc)
        print("reading file: {}".format(birdLoc))
        mfcc_samples = DomainManipulation.mfcc(y,50).T
        
        sfa = mdp.nodes.SFANode(output_dim=1)
        sfa.train(mfcc_samples)
        sfa.stop_training()
        sfa_component = sfa.sf.T
        sfa_samples = sfa(mfcc_samples)

        sfa_file_name_order.append(fileName)
        sfa_components.append(sfa_component.flatten())
        
        hmmModel = GMMHMM(n_components=2, n_mix=2)
        ns, blah = sfa_samples.shape
#         hmmModel.fit(sfa_samples[:,0].reshape((ns,1)))
#         logP, labels = hmmModel.decode(sfa_samples[:,0].reshape((ns,1)))
        hmmModel.fit(sfa_samples)
        logP, labels = hmmModel.decode(sfa_samples)
        labels = labels.flatten()
        
        hmm_sfa_corr.append(np.corrcoef(labels, sfa_samples.flatten())[0,1])
    except:
        print("fail")
        continue

    os.makedirs(os.path.join(targetLoc,"0"), exist_ok=True)
    os.makedirs(os.path.join(targetLoc,"1"), exist_ok=True)
    DataLoader.splitByLabels(y, labels, targetLoc)

    plt.subplot(211)
    plt.plot(range(sfa_samples.shape[0]), sfa_samples[:,0], 'b')
    plt.plot(range(sfa_samples.shape[0]), labels, 'r')

    plt.subplot(212)
    plt.plot(range(sfa_samples.shape[0]), DomainManipulation.rollingMean(y, 0, samples=sfa_samples.shape[0]), 'r')

    plt.savefig(targetLoc+".png")
    plt.clf()
sfa_components = np.array(sfa_components)

reading file: data/train_audio\aldfly\XC134874.mp3
reading file: data/train_audio\aldfly\XC135454.mp3
reading file: data/train_audio\aldfly\XC135455.mp3
reading file: data/train_audio\aldfly\XC135456.mp3
reading file: data/train_audio\aldfly\XC135457.mp3
reading file: data/train_audio\aldfly\XC135459.mp3
reading file: data/train_audio\aldfly\XC135460.mp3
reading file: data/train_audio\aldfly\XC135883.mp3
reading file: data/train_audio\aldfly\XC137570.mp3
reading file: data/train_audio\aldfly\XC138639.mp3
reading file: data/train_audio\aldfly\XC139577.mp3
reading file: data/train_audio\aldfly\XC140298.mp3
reading file: data/train_audio\aldfly\XC142065.mp3
reading file: data/train_audio\aldfly\XC142066.mp3
reading file: data/train_audio\aldfly\XC142067.mp3
reading file: data/train_audio\aldfly\XC142068.mp3
reading file: data/train_audio\aldfly\XC142329.mp3
reading file: data/train_audio\aldfly\XC144672.mp3
reading file: data/train_audio\aldfly\XC144674.mp3
reading file: data/train_audio\

In [3]:
print(sfa_components.shape)
print((sfa_file_name_order))
print((hmm_sfa_corr))

(95, 50)
['XC134874.mp3', 'XC135454.mp3', 'XC135455.mp3', 'XC135456.mp3', 'XC135457.mp3', 'XC135459.mp3', 'XC135460.mp3', 'XC135883.mp3', 'XC137570.mp3', 'XC138639.mp3', 'XC139577.mp3', 'XC140298.mp3', 'XC142065.mp3', 'XC142066.mp3', 'XC142067.mp3', 'XC142068.mp3', 'XC142329.mp3', 'XC144672.mp3', 'XC144674.mp3', 'XC144675.mp3', 'XC154310.mp3', 'XC154449.mp3', 'XC157462.mp3', 'XC167789.mp3', 'XC167791.mp3', 'XC16967.mp3', 'XC178480.mp3', 'XC178481.mp3', 'XC178485.mp3', 'XC179417.mp3', 'XC179600.mp3', 'XC180091.mp3', 'XC181484.mp3', 'XC182735.mp3', 'XC188377.mp3', 'XC188432.mp3', 'XC188434.mp3', 'XC188813.mp3', 'XC189042.mp3', 'XC189043.mp3', 'XC189045.mp3', 'XC189262.mp3', 'XC189263.mp3', 'XC189264.mp3', 'XC189265.mp3', 'XC189267.mp3', 'XC189268.mp3', 'XC189269.mp3', 'XC189589.mp3', 'XC193116.mp3', 'XC194087.mp3', 'XC194088.mp3', 'XC195541.mp3', 'XC195542.mp3', 'XC241646.mp3', 'XC244458.mp3', 'XC244459.mp3', 'XC245645.mp3', 'XC247799.mp3', 'XC247801.mp3', 'XC253455.mp3', 'XC2628.mp3', '

In [4]:
import DataLoader,DomainManipulation

segregateLoc = "data/audiosegregation-0.8"
targetdir = 'data/birdseperation-sfa-5-birds'

bird_names = os.listdir(traindir)

pca = PCA(n_components=1)
sfa_pca = pca.fit_transform(sfa_components)

sfa_pca.flatten()*hmm_sfa_corr
sfa_file_name_order

for name in bird_names[:5]:
    writepath = os.path.join(segregateLoc, name)
    readpath = os.path.join(targetdir, name)
    os.makedirs(os.path.join(writepath,"0"), exist_ok=True)
    os.makedirs(os.path.join(writepath,"1"), exist_ok=True)
    
    for filedir, sfa_pca_sign, hmm_sfa_sign in zip(sfa_file_name_order, sfa_pca, hmm_sfa_corr):
        full_file_dir = os.path.join(readpath, filedir)
        print(filedir, full_file_dir)
        
        if np.abs(hmm_sfa_sign)>0.8:
            if sfa_pca_sign*hmm_sfa_sign>0:
                DataLoader.copyAllFilesDirtoDir(os.path.join(full_file_dir,"0"),os.path.join(writepath,"0"))
                DataLoader.copyAllFilesDirtoDir(os.path.join(full_file_dir,"1"),os.path.join(writepath,"1"))
            else:
                DataLoader.copyAllFilesDirtoDir(os.path.join(full_file_dir,"0"),os.path.join(writepath,"1"))
                DataLoader.copyAllFilesDirtoDir(os.path.join(full_file_dir,"1"),os.path.join(writepath,"0"))
        
    DataLoader.mergeAudiosTo(os.path.join(writepath,"0"),writepath+"/0.wav")
    DataLoader.mergeAudiosTo(os.path.join(writepath,"1"),writepath+"/1.wav")

NameError: name 'sfa_components' is not defined